## Import relevant modules

In [ ]:
import numpy as np
import sep

In [ ]:
# additional setup for reading the test image and displaying plots
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 10.]

## Import UDF image using astropy

In [ ]:
#import image
fname = "f105w.fits"

#open using astropy.io
f105 = fits.open(fname)

#save as 2d numpy array
data = f105[0].data

#perform byte swap
data = data.byteswap(inplace=True).newbyteorder()

#close using astropy.io
f105.close()

## Show orginal image, save as PGN

In [ ]:
# show the image
m, s = np.mean(data), np.std(data)

f = plt.figure(figsize=(10,10))
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
f.savefig('udf_1.png')

## Use sep to measure background

In [ ]:
# measure a spatially varying background on the image
bkg = sep.Background(data)

## Get rms mean of background noise

In [ ]:
# get a "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

## Convert background to 2d numpy array

In [ ]:
# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()
# bkg_image = np.array(bkg) # equivalent to above

## Show image of background, save as PGN

In [ ]:
# show the background
f = plt.figure(figsize=(10,10))
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

#save image as png
f.savefig('udf_background1.png')

## Convert background noise to 2d numpy array

In [ ]:
# evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

## Show image of background noise, save as PGN

In [ ]:
# show the background noise
f = plt.figure(figsize=(10,10))
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

#save image as png
f.savefig('udf_background2.png')

## Correct data by subtracting background noise

In [ ]:
# subtract the background
data_sub = data - bkg

## Set detection treshhold to 1.5 times ro

In [ ]:
#set object detection threshold to be a constant value of 1.5ro
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

## Count detected objects

In [ ]:
# how many objects were detected
len(objects)

## Plot ellipses for each object, save as PGN

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    

#save image as png
plt.savefig('udf_ellipse.png')

## List available object data fields

In [ ]:
# available fields
objects.dtype.names

## Record flux, flux error data

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

## Show data for first 10 objects

In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))


## Make a histogram of the flux data

In [ ]:
# the histogram of the data
n, bins, patches = plt.hist(flux, 500, range=[-0.02,0.4])

plt.title('Histogram of Flux')
plt.show()
